In [43]:
#import re
import sys, codecs, optparse, glob, os
import string, yaml
import random, warnings
from ToolsMoodleXML2 import *

In [44]:
nKalaba="K3"
annee="17"
dossier="/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s"%(annee,nKalaba)
enonces=["Forme-Radical",
         "Forme-Traduction","Traduction-Radical",
#         "Forme-Traduction-MC","Traduction-Radical-MC",
#         "Forme-Proprietes",
         "Glose-Decoupe","Decoupe-Glose-MC","Decoupe-Glose"
        ]
enonces=[
         "Forme-Traduction","Traduction-Radical",
#         "Forme-Traduction-MC","Traduction-Radical-MC",
#         "Forme-Proprietes",
         "Proprietes-Forme",
#         "Glose-Decoupe",
         "Decoupe-Glose-MC",
#         "Decoupe-Glose"
        ]
theme="%s-%s-A1"%(annee,nKalaba)

#enonces=["Glose-Decoupe","Proprietes-Forme"]
#theme="%s-%s-A1"%(annee,nKalaba)

#enonces=["Theme","Version"]
#theme="Traductions"

#enonces=["Proprietes-Forme"]
#theme="%s-%s-B2"%(annee,nKalaba)

#enonces=["Ordre"]
#theme="Syntagmes"

In [45]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [46]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

In [47]:
for enonce in enonces:
    yamlFichier=base+theme+"-"+enonce+".yaml"
    csvFichier=base+theme+".txt"
    boolEX=False
    titreNoEX=False
            
    with open(yamlFichier, 'r') as stream:
        configuration=yaml.load(stream)

    with open(csvFichier, 'r') as file:
        donneesLignes=file.readlines()

    donnees=[]
    if boolEX or donneesLignes[0].startswith("#EX;"):
    #    print "avec #EX;"
        donneesEX()
    else:
    #    print "sans #EX;"
        donneesNoEX()

        
    categorie=configuration[u"Catégorie"]
    #donnees=configuration[u"Données"]
    consigne=configuration[u"Boucle"][u"contenu"]
    boucleWrap=[configuration[u"Boucle"][u"début"],configuration[u"Boucle"]["fin"]]
    boucleCLOZE=configuration[u"Boucle"][u"Cloze"]
    globalWrap=[configuration[u"Décor"][u"fond"]+configuration[u"Décor"][u"entete"]+configuration[u"Global"][u"début"],configuration[u"Global"]["fin"]]
    if "Conclusion" in configuration:
        conclusion=[configuration[u"Conclusion"][u"début"],configuration[u"Conclusion"]["contenu"],configuration[u"Conclusion"]["fin"]]
        conclusionCLOZE=configuration[u"Conclusion"][u"Cloze"]
    else:
        conclusion=["","",""]
        conclusionCLOZE=[]
    questions=XMLSeries(categorie)
    questionsConsigne=ClozeConsigne(consigne,boucleWrap=boucleWrap,globalWrap=globalWrap,conclusion=conclusion)
    serie=ClozeSerie(configuration[u"Boucle"][u"Cloze"],conclusionCLOZE)

    for exercices in donnees:
    #    print "exercices",exercices
        for exercice in exercices:
    #        print "exercice",exercice
            exerciceBoucle=exercices[exercice]["boucle"]
            exerciceConclusion=exercices[exercice]["conclusion"]
            serie.addExercice(Exercice(exerciceBoucle,exerciceConclusion,exercice.decode('utf8')))

    for exercice in serie.makeSerie(questionsConsigne):
#        print exercice.corps
#        print
        questions.addExercice(exercice)

    xmlOutput=MoodleXML()
    xmlOutput.addQuiz(questions.getSeries())

    xmlFichier=yamlFichier.split(".")[0]+".xml"

    with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
    #    print [questions.getClozes()]
        output.write(xmlOutput.getXML())

1 2 3 4 5 6 7 8 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52 53 55 56 57 58 59 60 61 62 64 65 66 67 68 69 70 71 73 74 75 76 77 78 79 80 82 83 84 85 86 87 88 89 91 92 93 94 95 96 97 98 100 101 102 103 104 105 106 107 109 110 111 112 113 114 115 116 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 280 281 282 283 284 285 286 287 288 289 290 291 292 293 

In [48]:
#xmlOutput.getXML()